This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Libraries
https://huggingface.co/docs/transformers/training

## Install

In [ ]:
!pip install datasets
!pip install transformers
!pip install opacus

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 136 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 29.5 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 32.8 MB/s 
     |████████████████████████████████| 127 kB 36.5 MB/s 
     |████████████████████████████████| 271 kB 35.7 MB/s 
     |████████████████████████████████| 144 kB 37.6 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.4 MB/s 
 

## Import

In [ ]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence
import gc
from opacus.utils.batch_memory_manager import BatchMemoryManager

import warnings
warnings.filterwarnings("ignore")

## Set drive if needed

In [ ]:
from google.colab import drive
import os

use_drive = True

if use_drive:
  drive.mount('/content/drive')
  COLABROOTDIR="/content/drive/My Drive/Projects/DP/"
  os.environ["COLABROOTDIR"] = COLABROOTDIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## [Check GPU footprint](https://stackoverflow.com/questions/59789059/gpu-out-of-memory-error-message-on-google-colab)

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed

def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
if len(GPUs) > 0:
  gpu = GPUs[0]
  printm()
else:
  print("No running GPU")

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=bc7f8a4b59259ec0d83fbae7621f0d1f2dd389f04eeea657867f322791c6c860
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 11.9 GB  |     Proc size: 798.6 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total     11441MB


## Get device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Dataset

## Download

First, we need to download the dataset. https://huggingface.co/datasets/tweets_hate_speech_detection

In [ ]:
from datasets import load_dataset, load_from_disk

# if you have processed and saved the dataset
dataset = load_from_disk(COLABROOTDIR + 'tweet-dataset')

# tweets_dataset = load_dataset('tweets_hate_speech_detection')

# for key in tweets_dataset.keys():
#   print(key, tweets_dataset[key].shape)

In [ ]:
num_labels = 2
# tweets_datasets = tweets_dataset['train'].train_test_split(test_size=0.1)
# tweets_datasets = tweets_datasets.rename_column("tweet", "text")

# tweets_datasets

In [ ]:
# lengths = []
# for i in ['train', 'test']:
#   for item in tweets_datasets[i]:
#     lengths.append(len(item['text']))

In [ ]:
import pandas as pd
# df = pd.DataFrame({'Lengths':lengths})
# df.describe()

## Gender swapping
Generate couple of test dataset versions switching gender based pronouns

In [ ]:
import re
male2female=u"""maleS femaleS, maleness femaleness,
him her, himself herself, his her, his hers, he she,
Mr Mrs, Mister Missus, Ms Mr, Master Miss, Master Mistress,
uncleS auntS, nephewS nieceS, sonS daughterS, grandsonS granddaughterS,
brotherS sisterS, man woman, men women, boyS girlS, paternal maternal,
grandfatherS grandmotherS,
husband wife, husbands wives, fatherS motherS, bridegroomS brideS, widowerS widowS,
KingS QueenS,PrinceS PrincessES,
Lord Lady, Lords Ladies,ladS lassES, sir madam, gentleman lady, gentlemen ladies,
godS goddessES, heroS heroineS, landlord landlady, landlords landladies, 
manservantS maidservantS, actorS actressES,
boyfriendS girlfriendS, dogS bitchES, daddy mommy, dadS momS"""
 
re_newline=re.compile(r",[ \n]*")
male2female_splitted=[ token.split(" ") for token in re_newline.split(male2female) ]
 
re_plural=re.compile("E*S$")
re_ES=re.compile("ES$")
 
def gen_pluralize(m,f):
# do plurals first 
  yield re_plural.sub("",m),re_plural.sub("",f)
  yield re_ES.sub("es",m),re_ES.sub("es",f)
  yield re_plural.sub("s",m),re_plural.sub("s",f)
 
def gen_capitalize_pluralize(m,f):
  for m,f in gen_pluralize(m,f):
    yield m.capitalize(), f.capitalize()
    yield m,f

# converts male pronouns to female and female to male
def gen_switch(male_to_female=True, female_to_male=True):
  switch={}
  words=[]

  for male,female in male2female_splitted:
    for xy, xx in gen_capitalize_pluralize(male,female):
    # for xy, xx in gen_pluralize(male,female):
      if male_to_female and xy not in switch: 
        switch[xy]=xx
        words.append(xy)
      if female_to_male and xx not in switch: 
        switch[xx]=xy
        words.append(xx)

  words="|".join(words)
  re_word = re.compile(r"\b("+words+r")\b")
  return re_word, switch

def rev_gender(text, re_word, switch):
  text=re_word.split(text)
  return "".join([ word+switch[gen] for word,gen in zip(text[::2],text[1::2])]) + text[-1]

In [ ]:
re_word_both, switch_both = gen_switch(male_to_female=True, female_to_male=True)
re_word_f2m, switch_f2m = gen_switch(male_to_female=False, female_to_male=True)
re_word_m2f, switch_m2f = gen_switch(male_to_female=True, female_to_male=False)

In [ ]:
# dataset = tweets_datasets
# dataset['test'] = dataset['test'].shuffle(seed=2022).shard(5, index=0)

In [ ]:
# dataset['test_gender_swapped'] = dataset['test']
# dataset['test_male_to_female'] = dataset['test']
# dataset['test_female_to_male'] = dataset['test']

# for index in tqdm(range(len(dataset['test']['text']))):
#     text = dataset['test']['text'][index]

#     dataset['test_gender_swapped']['text'][index] = rev_gender(text, re_word_both, switch_both)
#     dataset['test_female_to_male']['text'][index] = rev_gender(text, re_word_f2m, switch_f2m)
#     dataset['test_male_to_female']['text'][index] = rev_gender(text, re_word_m2f, switch_m2f)

  0%|          | 0/3197 [00:00<?, ?it/s]

In [ ]:
# save the dataset in drive since it takes much time to do this processing
# dataset.save_to_disk(COLABROOTDIR + 'tweet-dataset')

## Tokenizer

In [ ]:
from transformers import BertConfig, BertTokenizer

model_name = "bert-base-uncased"
config = BertConfig.from_pretrained(
    model_name,
    num_labels=2,
)
tokenizer = BertTokenizer.from_pretrained(
    model_name,
    do_lower_case=True,
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

## Prepare the data
Before we begin training, we need to preprocess the data and convert it to the format our model expects.

(Note: it'll take 5-10 minutes to run on a laptop)

In [ ]:
MAX_SEQ_LENGTH = 256

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=MAX_SEQ_LENGTH, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
# str value gets error during data loading step
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
# select a smaller subset for faster debugging
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=2022).select(range(5000))
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

test_swapped_dataset = tokenized_datasets['test_gender_swapped']
test_m2f_dataset = tokenized_datasets['test_male_to_female']
test_f2m_dataset = tokenized_datasets['test_female_to_male']

# Model

BERT (Bidirectional Encoder Representations from Transformers) is a state of the art approach to various NLP tasks. It uses a Transformer architecture and relies heavily on the concept of pre-training.

We'll use a pre-trained BERT-base model, provided in huggingface [transformers](https://github.com/huggingface/transformers) repo. It gives us a pytorch implementation for the classic BERT architecture, as well as a tokenizer and weights pre-trained on a public English corpus (Wikipedia).

Please follow these [installation instrucitons](https://github.com/huggingface/transformers#installation) before proceeding.

In [ ]:
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
from transformers import BertForSequenceClassification

def load_pretrained_model(model_name, config):
    model = BertForSequenceClassification.from_pretrained(model_name, config=config)

    trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
    total_params = 0
    trainable_params = 0

    for p in model.parameters():
      p.requires_grad = False
      total_params += p.numel()

    for layer in trainable_layers:
      for p in layer.parameters():
          p.requires_grad = True
          trainable_params += p.numel()

    print(f"Total parameters count: {total_params}") # ~108M
    print(f"Trainable parameters count: {trainable_params}") # ~7M

    return model

In [ ]:
model = load_pretrained_model(model_name, config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Total parameters count: 117163780
Trainable parameters count: 7680002


# Data loader

In [ ]:
BATCH_SIZE = 16

# needed for DP training
MAX_PHYSICAL_BATCH_SIZE = 2

In [ ]:
# train_dataloader = get_dataloader(small_train_dataset, BATCH_SIZE)
# test_dataloader = get_dataloader(small_eval_dataset, BATCH_SIZE)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE)

test_swapped_dataloader = DataLoader(test_swapped_dataset, batch_size=BATCH_SIZE)
test_m2f_dataloader = DataLoader(test_m2f_dataset, batch_size=BATCH_SIZE)
test_f2m_dataloader = DataLoader(test_f2m_dataset, batch_size=BATCH_SIZE)

# Training

## Utils

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# https://huggingface.co/docs/datasets/metrics
def calculate_result(labels, preds):
    return {
        'accuracy': np.round(accuracy_score(labels, preds), 4),
        'f1': np.round(f1_score(labels, preds), 4),
        'auc': np.round(roc_auc_score(labels, preds), 4)
    }

def evaluate(model, test_dataloader):    
    model.eval()

    losses, total_preds, total_labels = [], [], []
    
    for batch in test_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        
        preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
        labels = inputs['labels'].detach().cpu().numpy()
        
        losses.append(loss.item())
        total_preds.extend(preds)
        total_labels.extend(labels)
    
    model.train()
    return np.mean(losses), calculate_result(total_labels, total_preds), total_preds

def train(model, train_dataloader, epoch):
  losses, total_preds, total_labels = [], [], []
  progress_bar = tqdm(range(len(train_dataloader)), desc=f'Epoch {epoch}')

  for step, data in enumerate(train_dataloader):
    optimizer.zero_grad()

    inputs = {k: v.to(device) for k, v in data.items()}
    outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

    targets = data['labels'].to(device, dtype = torch.long)
    # loss = loss_function(outputs.logits, targets)
    loss = outputs[0]

    loss.backward()
    optimizer.step()

    losses.append(loss.item())

    preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
    labels = targets.detach().cpu().numpy()
    total_preds.extend(preds)
    total_labels.extend(labels)
    
    progress_bar.update(1)
    progress_bar.set_postfix(
        loss=np.round(np.mean(losses), 4), f1=np.round(f1_score(total_labels, total_preds), 4)
    )
          

  train_loss = np.mean(losses)
  train_result = calculate_result(np.array(total_labels), np.array(total_preds))

  return train_loss, train_result

In [ ]:
# Define optimizer
LEARNING_RATE = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

## Loop

In [ ]:
gc.collect()
EPOCHS = 3
# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train().to(device)

for epoch in range(1, EPOCHS+1):
    train_loss, train_result = train(model, train_dataloader, epoch)

    eval_loss, eval_result, eval_preds = evaluate(model, test_dataloader)

    print(
      f"Epoch: {epoch} | "
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Eval loss: {eval_loss:.3f} | "
      f"Eval result: {eval_result} | "
    )

Epoch 1:   0%|          | 0/1599 [00:00<?, ?it/s]

Epoch: 1 | Train loss: 0.170 | Train result: {'accuracy': 0.9425, 'f1': 0.4981, 'auc': 0.6938} |
Eval loss: 0.140 | Eval result: {'accuracy': 0.9504, 'f1': 0.6139, 'auc': 0.7778} | 


Epoch 2:   0%|          | 0/1599 [00:00<?, ?it/s]

Epoch: 2 | Train loss: 0.140 | Train result: {'accuracy': 0.9509, 'f1': 0.5757, 'auc': 0.7295} |
Eval loss: 0.122 | Eval result: {'accuracy': 0.9585, 'f1': 0.6563, 'auc': 0.7833} | 


Epoch 3:   0%|          | 0/1599 [00:00<?, ?it/s]

Epoch: 3 | Train loss: 0.134 | Train result: {'accuracy': 0.9546, 'f1': 0.6064, 'auc': 0.7425} |
Eval loss: 0.126 | Eval result: {'accuracy': 0.9548, 'f1': 0.5854, 'auc': 0.7292} | 


In [ ]:
swap_loss, swap_result, swap_preds = evaluate(model, test_swapped_dataloader)
m2f_loss, m2f_result, m2f_preds = evaluate(model, test_m2f_dataloader)
f2m_loss, f2m_result, f2m_preds = evaluate(model, test_f2m_dataloader)

In [ ]:
print(swap_loss, swap_result)
print(m2f_loss, m2f_result)
print(f2m_loss, f2m_result)

0.126207670874428 {'accuracy': 0.9548, 'f1': 0.5854, 'auc': 0.7292}
0.126207670874428 {'accuracy': 0.9548, 'f1': 0.5854, 'auc': 0.7292}
0.126207670874428 {'accuracy': 0.9548, 'f1': 0.5854, 'auc': 0.7292}


In [ ]:
preds_df = pd.DataFrame({'text':tokenizer.batch_decode(eval_dataset['input_ids'], skip_special_tokens=True), 
  'labels':eval_dataset['labels'], 'eval_pred':eval_preds,'swap_pred': swap_preds,
  'm2f_pred': m2f_preds,'f2m_pred': f2m_preds
})
preds_df.to_csv(COLABROOTDIR + 'results.csv', index=False)

## Analyze

In [ ]:
def anaylyze_mispredicts(df):
  mispredicted = df[df['eval_pred']!=df['labels']]
  mispredicted_females = mispredicted[mispredicted['has_female']].shape[0]
  has_females = df[df['has_female']].shape[0]
  print(f'Mispredicted tweets that mention female pronouns {mispredicted_females}')
  print(f'Tweets that mention female pronouns {has_females}.\nPercentage {mispredicted_females*100 / has_females}\n')

  mispredicted_males = mispredicted[mispredicted['has_male']].shape[0]
  has_males = df[df['has_male']].shape[0]
  print(f'Mispredicted tweets that mention male pronouns {mispredicted_females}')
  print(f'Tweets that mention male pronouns {has_males}.\nPercentage {mispredicted_males*100 / has_males}')

def anaylyze_predicts(df):
  predicted = df[df['eval_pred']==df['labels']]
  predicted_females = predicted[predicted['has_female']].shape[0]
  has_females = df[df['has_female']].shape[0]
  print(f'Predicted tweets that mention female pronouns {predicted_females}')
  print(f'Tweets that mention female pronouns {has_females}.\nPercentage {predicted_females*100 / has_females}\n')

  predicted_males = predicted[predicted['has_male']].shape[0]
  has_males = df[df['has_male']].shape[0]
  print(f'Predicted tweets that mention male pronouns {predicted_females}')
  print(f'Tweets that mention male pronouns {has_males}.\nPercentage {predicted_males*100 / has_males}')

In [ ]:
preds_df['has_male'] = preds_df['text'].apply(lambda x: re_word_m2f.search(x) is None)
print(preds_df['has_male'].describe())

preds_df['has_female'] = preds_df['text'].apply(lambda x: re_word_f2m.search(x) is None)
print(preds_df['has_female'].describe())

mispredicted = preds_df[preds_df['eval_pred']!=preds_df['labels']]

count     6393
unique       2
top       True
freq      5771
Name: has_male, dtype: object
count     6393
unique       2
top       True
freq      5967
Name: has_female, dtype: object


In [ ]:
anaylyze_mispredicts(preds_df)
print('\n')
anaylyze_predicts(preds_df)

Mispredicted tweets that mention female pronouns 250
Tweets that mention female pronouns 5967.
Percentage 4.189710072063013

Mispredicted tweets that mention male pronouns 250
Tweets that mention male pronouns 5771.
Percentage 4.349332871252816


Predicted tweets that mention female pronouns 5717
Tweets that mention female pronouns 5967.
Percentage 95.81028992793699

Predicted tweets that mention male pronouns 5717
Tweets that mention male pronouns 5771.
Percentage 95.65066712874719


# Private Training

In [ ]:
EPOCHS = 3
EPSILON = 7.5
# DELTA = 1 / len(train_dataloader) # Parameter for privacy accounting. Probability of not achieving privacy guarant
DELTA = 1e-5
NOISE_MULTIPLIER = 0.05
LEARNING_RATE = 1e-3
MAX_GRAD_NORM = 1

In [ ]:
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

# criterion
# loss_function = torch.nn.CrossEntropyLoss()

## Privacy Engine

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

In [ ]:
# model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
#     module=model,
#     optimizer=optimizer,
#     data_loader=train_dataloader,
#     target_delta=DELTA,
#     target_epsilon=EPSILON, 
#     epochs=EPOCHS,
#     max_grad_norm=MAX_GRAD_NORM,
# )

model, optimizer, train_dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    noise_multiplier=NOISE_MULTIPLIER,
    max_grad_norm=MAX_GRAD_NORM,
    poisson_sampling=False,
)

## Train

In [ ]:
import gc
gc.collect()

605

In [ ]:
def dp_train(model, train_dataloader, epoch):
  losses, total_preds, total_labels = [], [], []
  

  with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        progress_bar = tqdm(range(len(memory_safe_data_loader)), desc=f'Epoch {epoch}')

        for step, data in enumerate(memory_safe_data_loader):
          optimizer.zero_grad()

          inputs = {k: v.to(device) for k, v in data.items()}
          outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

          targets = data['labels'].to(device, dtype = torch.long)
          # loss = loss_function(outputs.logits, targets)
          loss = outputs[0]

          loss.backward()
          optimizer.step()

          losses.append(loss.item())

          preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
          labels = targets.detach().cpu().numpy()
          total_preds.extend(preds)
          total_labels.extend(labels)
          
          progress_bar.update(1)
          progress_bar.set_postfix(
              loss=np.round(np.mean(losses), 4), f1=np.round(f1_score(total_labels, total_preds), 4)
          )

  train_loss = np.mean(losses)
  train_result = calculate_result(np.array(total_labels), np.array(total_preds))

  return train_loss, train_result

In [ ]:
for epoch in range(1, EPOCHS+1):
    train_loss, train_result = dp_train(model, train_dataloader, epoch)

    eps = privacy_engine.get_epsilon(DELTA)
    eval_loss, eval_result, eval_preds = evaluate(model, test_dataloader)

    print(
      f"Epoch: {epoch} | "
      f"ɛ: {eps:.2f} |"
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Eval loss: {eval_loss:.3f} | "
      f"Eval result: {eval_result} | "
    )

Epoch 1:   0%|          | 0/1599 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
swap_loss, swap_result, swap_preds = evaluate(model, test_swapped_dataloader)
m2f_loss, m2f_result, m2f_preds = evaluate(model, test_m2f_dataloader)
f2m_loss, f2m_result, f2m_preds = evaluate(model, test_f2m_dataloader)

In [ ]:
swap_result, m2f_result, f2m_result

({'accuracy': 0.9526, 'auc': 0.711, 'f1': 0.5538},
 {'accuracy': 0.9526, 'auc': 0.711, 'f1': 0.5538},
 {'accuracy': 0.9526, 'auc': 0.711, 'f1': 0.5538})

In [ ]:
dp_preds_df = pd.DataFrame({'text':tokenizer.batch_decode(eval_dataset['input_ids'], skip_special_tokens=True), 
  'labels':eval_dataset['labels'], 'eval_pred':eval_preds,'swap_pred': swap_preds,
  'm2f_pred': m2f_preds,'f2m_pred': f2m_preds
})

In [ ]:
if use_drive:
  dp_preds_df.to_csv(COLABROOTDIR + 'results_dp.csv', index=False)

## Analyze

In [ ]:
dp_preds_df['has_male'] = dp_preds_df['text'].apply(lambda x: re_word_m2f.search(x) is None)
print(dp_preds_df['has_male'].describe())

dp_preds_df['has_female'] = dp_preds_df['text'].apply(lambda x: re_word_f2m.search(x) is None)
print(dp_preds_df['has_female'].describe())

count     6393
unique       2
top       True
freq      5771
Name: has_male, dtype: object
count     6393
unique       2
top       True
freq      5967
Name: has_female, dtype: object


In [ ]:
anaylyze_mispredicts(dp_preds_df)
print('\n')
anaylyze_predicts(dp_preds_df)

Mispredicted tweets that mention female pronouns 260
Tweets that mention female pronouns 5967.
Percentage 4.357298474945534

Mispredicted tweets that mention male pronouns 260
Tweets that mention male pronouns 5771.
Percentage 4.574597123548778


Predicted tweets that mention female pronouns 5707
Tweets that mention female pronouns 5967.
Percentage 95.64270152505446

Predicted tweets that mention male pronouns 5707
Tweets that mention male pronouns 5771.
Percentage 95.42540287645122


Predicted tweets that mention female pronouns 5707
Tweets that mention female pronouns 5967.
Percentage 95.64270152505446

Predicted tweets that mention male pronouns 5707
Tweets that mention male pronouns 5771.
Percentage 95.42540287645122
